In [3]:
import warnings
warnings.filterwarnings('ignore')

Inside __/pytorch/images__, you should find a __/style_images__ folder and a __/sample_content_images__ folder.

- Inside the __/style_images__ folder is an image of Vangogh painting (titled, __sample_vangogh.jpg__) that we will be using as our sample style image. 
- Inside the __/sample_content_images__ folder, which we will use solely for the purposes of testing locally, there are 4 sample images that we will use to apply the style onto. (images are titled, __sample_%1.jpg__)

The interactive notebook (__style_transfer_interactive.ipynb__) and the script (__style_transfer_script.py__) will using the above directories.

## (Optional) Tuning the style transfer hyperparameters interactively

The first thing we want to do is to test the style transfer scripts locally and make sure the hyper parameters are set appropriately. This will be done using the __style_transfer_interactive.ipynb__ notebook.

Open the notebook __/pytorch/style_transfer_interactive.ipynb__ and tune the variables following variables as desired: 

- STYLE_WEIGHT = 10**8
- CONTENT_WEIGHT = 10**0
- NUM_STEPS = 300

*the defaults shown above _tend_ to work nicely for most images.

## Testing the style transfer script locally

Lets make sure that our style transfer script is running correctly on our local machine. First we need to create the directory to store the output images.

In [ ]:
%%bash
mkdir pytorch/images/sample_output_images

If you changed the hyperparameters in the above section and would like to apply it, you can use the following variables: `--style-weight`, `--content-weight`, and/or `--num-steps`

In [ ]:
%%bash
cd pytorch &&
    python style_transfer_script.py \
    --style-image ./images/style_images/sample_vangogh.jpg \
    --content-image-dir ./images/sample_content_images \
    --content-image-list sample_0.jpg,sample_1.jpg \
    --output-image-dir ./images/sample_output_images \
    --style-weight 100000000 \
    --content-weight 1 \
    --num-steps 300

We can now inspect the output directory to make sure that the output images are there.

In [ ]:
%%bash
ls pytorch/images/sample_output_images

Lets see what one of those images look like:

![sample_0](pytorch/images/sample_output_images/sample_0.jpg)

## Setup your video for batch style transfer

### Download Video: 
The first thing we need to do is to download a video that we would like to apply style transfer onto. To do so, we'll be using __youtube-dl__, a simple open source command line utility to download a video from YouTube. We'll keep the downloaded video in a new folder: __pytorch/video__. 

First we'll download __youtube-dl__ and use it to download a video. In this notebook, we're going to download a short 8 minute video of someone teaching us how to raise chickens. In the download command, we'll also use the flag `-f 22` to tell __youtube_dl__ that we want to download an mp4 format with dimensions 720x1280. 

*Feel free to manually place your own video (mp4) file into that directory instead of using __youtube-dl__ to download one...

In [ ]:
%%bash
mkdir pytorch/video

In [ ]:
%%bash
sudo -H pip install --upgrade youtube-dl &&
    cd pytorch/video &&
    youtube-dl -f 22 -o "chicken.%(ext)s" "https://www.youtube.com/watch?v=D23sMvVnrow"

### Pre-process video with ffmpeg

Next we need to use __ffmpeg__ to extract the audio file, and break up the frames of the video into separate individual images.

In [ ]:
%%bash 
cd pytorch/video &&
    ffmpeg -i chicken.mp4 -vn -acodec copy chicken.aac

In [ ]:
%%bash
cd pytorch/images/ &&
    mkdir chicken_frames && cd chicken_frames &&
    ffmpeg -i ../../video/chicken.mp4 %05d_chicken.jpg -hide_banner

At this point, our file system should have the following under the folder __pytorch_style_transfer__:
```md
├── images/
│   ├── chicken_content_images/ [<--new]
│   ├── sample_content_images/
│   ├── sample_output_images/
│   └── style_images/
├── video/
│   ├── chicken.mp4 [<--new]
│   └── chicken.avi [<--new]
├── style_transfer_interactive.ipynb
└── style_transfer_script.ipynb
```

Now that we have all the frames of the video stored seperately as images, we are ready to scale out and try scoring in the cloud.

## Testing our Style Transfer Script on Azure BatchAI

In this tutorial, we use Azure BatchAI to scale out of computation to multiple GPUs in Azure.

### Setting up your cluster

First, we need to create your cluster using the __azure/scripts/create_cluster.py__ script.

After running this command, go into the Azure portal and check that your cluster is successfully created.

In [1]:
%%bash 
python azure/scripts/create_cluster.py

Cluster state: resizing; Allocated: 5; Idle: 0; Unusable: 0; Running: 5; Preparing: 0; Leaving: 0


Keyring cache token has failed: No recommended backend was available. Install the keyrings.alt package if you want to use the non-recommended backends. See README.rst for details.


### Upload files into Azure Blob Storage

Now we need to upload the script and model files to the fileshare using the __azure/scripts/upload_files.py__ script. 

After running this command, go into the Azure portal or the Azure Storage Explorer and make sure that the files are correctly uploaded.

In [1]:
%%bash
python azure/scripts/upload_files.py \
    --style-transfer-script pytorch/style_transfer_script.py \
    --style-image pytorch/images/style_images/sample_vangogh.jpg \
    --content-images-dir pytorch/images/chicken_frames

### Create a Test job

Finally, lets test that our style transfer script works on the Batch AI cluster. Use the __azure/scripts/create_job.py__ script to kick off a job. 

NOTE - this command could take a while to execute. Remember that this will apply style transfer onto each frame of the video - this means that the code will optimize the style transfer loss function for every frame. 

After running this command, go into the Azure portal under your BatchAI account to make sure the job is sucessfully created.

In [2]:
%%bash
python azure/scripts/create_job.py

Created Job: job0_08_02_2018_193053
Created Job: job1_08_02_2018_193111
Created Job: job2_08_02_2018_193128
Created Job: job3_08_02_2018_193145
Created Job: job4_08_02_2018_193202
Created Job: job5_08_02_2018_193218
Created Job: job6_08_02_2018_193235
Created Job: job7_08_02_2018_193252
Created Job: job8_08_02_2018_193309
Created Job: job9_08_02_2018_193326


Keyring cache token has failed: No recommended backend was available. Install the keyrings.alt package if you want to use the non-recommended backends. See README.rst for details.


### Inspect the results

When the jobs finish running, you can use the Azure portal or Storage explorer to inspect the output images.

Inside your Blob Container, you should notice that a new directory with the datetime-stamp is created. Output images are stored there.

## Running it with Docker

After we've tested that the BatchAI jobs are successfully created, we now want to build a docker container and check that we can run the BatchAI job from a docker container.

First we need to build the docker image using the Dockerfile which will upload all Azure utility python files as well as the __create_job.py__ file into the image.

In [ ]:
%%bash
cd azure &&
    sudo docker build -t bai_job .

Then we need to test that the BatchAI job can be executed from the docker image we just built. Because the __create_job.py__ file requires many environment variables, we will use the __docker_run.sh__ script to pass in all the required environment variables for the image to successfully run locally. 

In [ ]:
%%bash
cd azure &&
    source docker_run.sh -t bai_job 

Finally, we have to publish the image to Dockerhub

In [ ]:
%%bash
cd azure &&
    sudo docker tag bai_job <your-dockerhub-username>:bai_job &&
    sudo docker push <your-dockerhub-username>:bai_job 

## Setting up ACI with Logic Apps

TODO